### Build the CSV of pedestal values (HV & NoHV) for FPGA 209 (PB 008) and FPGA 210 (PB 06)

In [30]:
import json
from pathlib import Path
import pandas as pd

In [31]:
base = Path("/home/lorena/Documents/PhD/EEEMCal_Analysis/Pedestal_Stability/Data/Pedestals")
files = {
    ("209", "008", "NoHV"): base / "103_PedestalCalib_pedecalib_20251017_132119.json",
    # ("209", "008", "NoHV"): base / "103_PedestalCalib_pedecalib_20251017_174324.json",
    ("209", "008", "HV"):   base / "103_PedestalCalib_pedecalib_20251017_174324.json",
    # ("209", "008", "HV"):   base / "103_PedestalCalib_pedecalib_20251017_132119.json",
    ("210", "06",  "NoHV"): base / "103_PedestalCalib_pedecalib_20251017_132121.json",
    ("210", "06",  "HV"):   base / "103_PedestalCalib_pedecalib_20251017_174322.json",
}

In [32]:
# Channels to exclude completely (1-based indexing)
exclude_channels = {19, 38, 57, 76, 95, 114, 133}

# Helper: load pede_values into a dict {channel_number: pedestal}
def load_pedestals(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)
    pede_values = data.get("pede_values", [])
    # Build 1..151 mapping (skip channel 0, and ignore >151 if present)
    mapping = {ch: pede_values[ch] for ch in range(1, min(len(pede_values), 152)) if ch not in exclude_channels}
    return mapping

In [33]:
# Build long dataframe: one row per (FPGA, PB, Channel, Condition, Pedestal)
rows = []
for (fpga, pb, cond), path in files.items():
    ped = load_pedestals(path)
    for ch, val in ped.items():
        rows.append({
            "FPGA": fpga,
            "PB": pb,
            "Channel": ch,
            "Condition": cond,
            "Pedestal": val
        })

df_long = pd.DataFrame(rows).sort_values(["FPGA", "PB", "Channel", "Condition"]).reset_index(drop=True)

In [34]:
# Pivot to get HV and NoHV columns side-by-side per FPGA/PB/Channel
df_wide = df_long.pivot_table(index=["FPGA", "PB", "Channel"], columns="Condition", values="Pedestal").reset_index()
# Ensure column order
if "HV" not in df_wide.columns: df_wide["HV"] = pd.NA
if "NoHV" not in df_wide.columns: df_wide["NoHV"] = pd.NA
df_wide = df_wide[["FPGA", "PB", "Channel", "NoHV", "HV"]].sort_values(["FPGA", "PB", "Channel"]).reset_index(drop=True)


In [35]:
# Save CSV
out_csv = base / "pedestals_HV_vs_NoHV_FPGA209_210.csv"
df_wide.to_csv(out_csv, index=False)

In [36]:
# Show first 20 rows in Jupyter
display(df_wide.head(20))

# Print output file path
print(f"✅ CSV saved to: {out_csv}")


Condition,FPGA,PB,Channel,NoHV,HV
0,209,008,1,101.0,81.0
1,209,008,2,96.0,79.0
2,209,008,3,95.0,79.0
3,209,008,4,95.0,78.0
4,209,008,5,97.0,81.0
5,209,008,6,95.0,79.0
6,209,008,7,95.0,81.0
7,209,008,8,95.0,79.0
8,209,008,9,115.0,118.0
9,209,008,10,94.0,81.0


✅ CSV saved to: /home/lorena/Documents/PhD/EEEMCal_Analysis/Pedestal_Stability/Data/Pedestals/pedestals_HV_vs_NoHV_FPGA209_210.csv


In [37]:
# Verify that each JSON has exactly one pedestal per channel (excluding skipped ones)

for (fpga, pb, cond), path in files.items():
    ped = load_pedestals(path)
    print(f"{ped}")
    unique_channels = len(ped)
    unique_values = len(set(ped.keys()))
    print(f"{fpga=} {pb=} {cond=}: {unique_channels} unique channels mapped")

    # sanity check
    if unique_channels != unique_values:
        print("⚠️  Channel index duplication detected!")
    else:
        print("✅  One-to-one mapping confirmed.")

    # quick check of possible duplicates or length mismatch
    if len(ped) != (151 - len(exclude_channels)):
        print(f"⚠️  Expected {151 - len(exclude_channels)} channels, found {len(ped)}")
    else:
        print("✅  Channel count matches expectation.")
    print("-"*50)

{1: 101.0, 2: 96.0, 3: 95.0, 4: 95.0, 5: 97.0, 6: 95.0, 7: 95.0, 8: 95.0, 9: 115.0, 10: 94.0, 11: 107.0, 12: 102.0, 13: 94.0, 14: 97.0, 15: 93.0, 16: 95.0, 17: 93.0, 18: 78.0, 20: 95.0, 21: 93.0, 22: 91.0, 23: 95.0, 24: 93.0, 25: 93.0, 26: 91.0, 27: 92.0, 28: 80.0, 29: 87.0, 30: 92.0, 31: 89.0, 32: 87.0, 33: 91.0, 34: 91.0, 35: 84.0, 36: 91.0, 37: 0.0, 39: 81.0, 40: 79.0, 41: 81.0, 42: 79.0, 43: 84.0, 44: 80.0, 45: 80.0, 46: 82.0, 47: 83.0, 48: 81.0, 49: 83.0, 50: 101.0, 51: 79.0, 52: 85.0, 53: 80.0, 54: 82.0, 55: 81.0, 56: 74.0, 58: 85.0, 59: 81.0, 60: 85.0, 61: 85.0, 62: 86.0, 63: 90.0, 64: 83.0, 65: 91.0, 66: 81.0, 67: 88.0, 68: 86.0, 69: 86.0, 70: 83.0, 71: 85.0, 72: 83.0, 73: 85.0, 74: 87.0, 75: 0.0, 77: 89.0, 78: 90.0, 79: 92.0, 80: 89.0, 81: 89.0, 82: 91.0, 83: 86.0, 84: 86.0, 85: 80.0, 86: 86.0, 87: 91.0, 88: 92.0, 89: 85.0, 90: 111.0, 91: 91.0, 92: 89.0, 93: 89.0, 94: 86.0, 96: 90.0, 97: 88.0, 98: 86.0, 99: 86.0, 100: 85.0, 101: 89.0, 102: 87.0, 103: 73.0, 104: 79.0, 105: 85.0